# Fig4-Bacteria spatial distributions

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from glob import glob
import matplotlib as mpl
from skimage.io import imread

In [4]:
%matplotlib qt

In [35]:
"""plot style"""
linewidth = 4
mpl.rc('axes', linewidth=linewidth)
mpl.rc('font', family='Arial')
fontsize = 24


colors = {'no_inj': [0.8, 0.8, 0.8],
         'mock': [0.4, 0.4, 0.4],
         'e.coli': [0, 0.4, 0],
         'complete': [0, 0.8, 0],
         'bacteria': [0.8, 0, 0.8]}

def style_axes(ax, fontsize=24):
    plt.minorticks_off()
    #ax.spines['top'].set_visible(False)
    #ax.spines['right'].set_visible(False)
    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    plt.tight_layout()
    
    return ax


In [9]:
def bin_aps(df, bins, ap_col='y'):
    _counts, bins = np.histogram(df.get(ap_col), bins)
    bins = bins[1:]
    partial_func = partial(get_ap_bin, bins=bins)
    binned_aps = df.get(ap_col).apply(partial_func)
    df[f'binned_{ap_col}'] = binned_aps.values
    
    return df

def get_ap_bin(this_ap, bins):
    this_bin = np.where(np.abs(this_ap - bins) == np.nanmin(np.abs(this_ap - bins)))[0][0]

    return this_bin

In [7]:
experiment_paths = [r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/2024_07_02-NP1029-Gal4_ecoli-hs-dtom_2hr']

In [19]:
"""params"""
df_name = 'bacteria.pkl'
method = 'bkg_sub_sum_data'

n_ap_bins = 14#22
ap_bins = np.linspace(0, 1, n_ap_bins + 1)
fb_shift = 0.12
ap = (ap_bins[:-1] / np.max(ap_bins[:-1]) - fb_shift) / (1 - fb_shift)

available_volume = np.array([3.49238605e+07, 6.53988130e+07, 1.01998991e+08, 1.25074541e+08,
       1.46105305e+08, 1.56498386e+08, 1.62622979e+08, 1.80517528e+08,
       1.77813337e+08, 1.90367745e+08, 1.81240248e+08, 1.79402863e+08,
       1.75391286e+08, 1.69207341e+08, 1.61747398e+08, 1.47125100e+08,
       1.39735655e+08, 1.09845337e+08, 9.37926961e+07, 6.89674743e+07,
       4.08586640e+07, 1.99802229e+07])

x = np.linspace(0, 1, n_ap_bins)
xp = np.linspace(0, 1, len(available_volume))
sampled_volume = np.interp(x, xp, available_volume)
sampled_volume = np.flip(sampled_volume)

all_ap_dists = np.zeros((4, n_ap_bins))
all_ap_dens = np.zeros((4, n_ap_bins))




In [20]:
counter = 0
for path in experiment_paths:
    larvae_dirs = glob(path + '/larva*')
    for larvae_dir in larvae_dirs:
        if larvae_dir == r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/2024_07_02-NP1029-Gal4_ecoli-hs-dtom_2hr/larva_2':
            continue
        
        if larvae_dir == r'/media/brandon/Data2/Brandon/fly_immune/Lightsheet_Z1/2024_07_02-NP1029-Gal4_ecoli-hs-dtom_2hr/larva_3':
            continue
        
        if len(glob(larvae_dir + '/' + df_name)) == 0: 
            continue
            
        df = pd.read_pickle(larvae_dir + '/' + df_name)
        func = lambda row: row.data[0].size
        sizes = df.apply(func, axis=1)
        df = df[sizes < 1e5]

        df['ap'] = (df.y.max() - df.y) / df.y.max()

        df = bin_aps(df, ap_bins, ap_col='ap')
        ap_col = 'ap'

        intens = df.get(method)
        single_cell_inten = np.median(intens)
        df['n_bacteria'] = np.clip(intens / single_cell_inten, a_min=1, a_max=np.inf).astype('int')
        
        """filter bacterial clusters by size"""
        #df = df[df.n_bacteria < 3]
        
        """plot number of bacteria vs ap axis"""
        binned_sum = df.get(['n_bacteria', f'binned_{ap_col}']).groupby(by=f'binned_{ap_col}').sum().values.flatten()
        ap_bin_numbers = sorted(df.get(f'binned_{ap_col}').unique())
        binned_concentration = binned_sum / sampled_volume * 1e12 / 1e7
        all_ap_dists[counter, ap_bin_numbers] = binned_sum
        all_ap_dens[counter, ap_bin_numbers] = binned_concentration 
        
        counter += 1
       

In [47]:
dpt_norm = np.array([0.07903883, 0.13227668, 0.16431015, 0.19564819, 0.24321526,
       0.32607098, 0.38470999, 0.45284919, 0.53319832, 0.5511665 ,
       0.55050679, 0.57733441, 0.56615243, 0.5690741 , 0.56713838,
       0.55950318, 0.55885232, 0.56597654, 0.59410658, 0.59004553,
       0.5520511 , 0.52517944, 0.48008338, 0.42171416, 0.39567167,
       0.3581711 , 0.33449375, 0.35740834, 0.35988537, 0.33202247,
       0.35888999, 0.35009875, 0.31940752, 0.33306071, 0.31320107,
       0.32884156, 0.28807233, 0.26897305, 0.27515221, 0.26254933,
       0.24853601, 0.2546311 , 0.24866373, 0.26167798, 0.26919918,
       0.25207219, 0.25698458, 0.25397448, 0.25178925, 0.25171474,
       0.24231979, 0.24563699, 0.24292123, 0.25452599, 0.25016401,
       0.255014  , 0.24383757, 0.25662023, 0.25803773, 0.24124198,
       0.26288938, 0.28658983, 0.28069496, 0.27422479, 0.28525263,
       0.28332698, 0.29394878, 0.27291115, 0.27597078, 0.30209175,
       0.2866481 , 0.32373916, 0.33786614, 0.33041474, 0.35924229,
       0.38451242, 0.43435841, 0.47938158, 0.53872952, 0.55393285,
       0.6082538 , 0.59959786, 0.60445236, 0.59490812, 0.6003198 ,
       0.58362359, 0.55703335, 0.59825418, 0.65505778, 0.65172646,
       0.68970665, 0.69248515, 0.65785197, 0.5632604 , 0.50314999,
       0.41424737, 0.34565377, 0.30370179, 0.24694057, 0.10405844])

dpt_sig = np.array([0.0315106 , 0.05301132, 0.07513997, 0.0998847 , 0.11253486,
       0.13619532, 0.15352773, 0.14574699, 0.19179522, 0.1573023 ,
       0.17313237, 0.16427777, 0.16994246, 0.17095068, 0.16307587,
       0.172179  , 0.17434761, 0.22212662, 0.22433747, 0.18545454,
       0.17736996, 0.19010708, 0.20994788, 0.16303315, 0.13060417,
       0.09801501, 0.1123612 , 0.08687323, 0.07451827, 0.08076324,
       0.1071621 , 0.11341535, 0.10879741, 0.15929675, 0.13994247,
       0.14417039, 0.14903977, 0.11146931, 0.10260015, 0.10099548,
       0.08420312, 0.07638019, 0.07660251, 0.12371982, 0.13636218,
       0.12551225, 0.1351894 , 0.1139981 , 0.09998717, 0.08922782,
       0.08240206, 0.08897153, 0.12400051, 0.11914447, 0.12555039,
       0.12422211, 0.12283066, 0.11759031, 0.13411745, 0.09351607,
       0.09851103, 0.14696117, 0.13686434, 0.13010025, 0.12941647,
       0.13327248, 0.10340321, 0.10861777, 0.08943369, 0.13314903,
       0.13319672, 0.12765173, 0.11964749, 0.13164771, 0.15649755,
       0.14607092, 0.15595638, 0.17228274, 0.20807284, 0.20715987,
       0.2264893 , 0.21731211, 0.23274834, 0.21190218, 0.22544983,
       0.2595738 , 0.24039126, 0.22102505, 0.20296925, 0.17732799,
       0.21518867, 0.26116908, 0.27392563, 0.27020937, 0.27885535,
       0.2657325 , 0.24348111, 0.215349  , 0.16729179, 0.06852597])

In [52]:
plt.figure(figsize=(8,4))
m = np.mean(all_ap_dens, axis=0)
s = np.std(all_ap_dens, axis=0)
u = m + s
l = m - s
plt.fill_between(ap, l, u, color=colors['bacteria'], alpha=0.5, zorder=100)
plt.plot(ap, m, 'm-', linewidth=4, zorder=102)
plt.xlabel('fraction of anterior-posterior axis', fontsize=24)
plt.ylabel('bacteria density \n($10^7$ cells / ml)', fontsize=24, color=colors['bacteria'])
plt.xlim(-fb_shift, 1)
ax = style_axes(plt.gca())
ax.tick_params(axis='y', labelcolor=colors['bacteria'])

ax2 = ax.twinx()
dpt_ap = np.linspace(-fb_shift, 1, len(dpt_norm))
dpt_l = dpt_norm - dpt_sig
dpt_u = dpt_norm + dpt_sig
#plt.fill_between(dpt_ap, dpt_l, dpt_u, color=colors['e.coli'], alpha=0.5, zorder=-1)
ax2.plot(dpt_ap, dpt_norm, linewidth=linewidth, color=colors['e.coli'], alpha=0.5, zorder=-1)
ax2.set_ylabel('normalized diptericin-gfp \nfluorescence intensity \n(fraction of max)', fontsize=fontsize, color=colors['e.coli'])
ax2.tick_params(axis='y', labelcolor=colors['e.coli'])
ax2 = style_axes(ax2)


In [54]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig4-Bacteria/bacteria_and_dpt_ap_dists_long.pdf')

## Dye analysis

In [60]:
dye_experiment_paths = [r'/media/brandon/Data1/Brandon/fly_immune/bb8/2024_06_27_NP1029-gal4_rhodamine/5min_exp30ms_power100_mag29pt5']

In [93]:
all_dye_dists = []
larva_thresh = 2000
n_ap_bins = 30
counter = 0
for path in dye_experiment_paths:
    larvae_dirs = sorted(glob(path + '/larva*'))
    for larvae_dir in larvae_dirs:
        file = glob(larvae_dir + '/Pos0/*.tif')
        if len(file) > 1:
            print(f'more than 1 tif in {larvae_dir}/Pos0')
            continue
        im = imread(file[0])
        im[im < larva_thresh] = 0
        this_dist = np.sum(im, axis=1)
        larva_mask = np.sum(im > 0, axis=1)
        first_id = np.where(larva_mask)[0][0]
        last_id = np.where(larva_mask)[0][-1]
        this_dist = this_dist[first_id:last_id]
        tmp_df = pd.DataFrame()
        tmp_df['y'] = np.arange(len(this_dist))
        tmp_df['inten'] = this_dist
        ap_bins = np.linspace(0, len(this_dist), n_ap_bins)
        tmp_df = bin_aps(tmp_df, ap_bins)
        tmp_df = tmp_df.groupby(by='binned_y').sum()
        
        all_dye_dists.append(tmp_df.inten.values)

all_dye_dists_arr = np.zeros((len(all_dye_dists), len(all_dye_dists[0])))
for i in range(len(all_dye_dists)):
    all_dye_dists_arr[i] = all_dye_dists[i]
    
all_dye_dists_arr /= 1e7

In [96]:
plt.figure()
for dd in all_dye_dists:
    x = np.linspace(-fb_shift, 1, len(dd))
    plt.plot(x, dd)
    

plt.figure(figsize=(8,4))
color = np.array([204., 85., 0.]) / 255
m = np.mean(all_dye_dists_arr, axis=0)
s = np.std(all_dye_dists_arr, axis=0)
l = m - s
u = m + s
x = np.linspace(-fb_shift, 1, all_dye_dists_arr.shape[1])
plt.fill_between(x, l, u, color=color, alpha=0.5, zorder=100)
plt.plot(x, m, color=color, linewidth=4, zorder=102)
ax = style_axes(plt.gca())
plt.xlabel('fraction of anterior-posterior axis', fontsize=24)
plt.ylabel('rhodamine \nfluorescence \nintensity (a.u.)', fontsize=24, color=color)
ax.tick_params(axis='y', labelcolor=color)

ax2 = ax.twinx()
dpt_ap = np.linspace(-fb_shift, 1, len(dpt_norm))
dpt_l = dpt_norm - dpt_sig
dpt_u = dpt_norm + dpt_sig
#plt.fill_between(dpt_ap, dpt_l, dpt_u, color=colors['e.coli'], alpha=0.5, zorder=-1)
ax2.plot(dpt_ap, dpt_norm, linewidth=linewidth, color=colors['e.coli'], alpha=0.5, zorder=-1)
ax2.set_ylabel('normalized diptericin-gfp \nfluorescence intensity \n(fraction of max)', fontsize=fontsize, color=colors['e.coli'])
ax2.tick_params(axis='y', labelcolor=colors['e.coli'])
ax2 = style_axes(ax2)

In [97]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig4-Bacteria/rhodamine_and_dpt_ap_dists_long.pdf')